In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.special import expit
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:

class LogisticRegressionWithEntropyRegularization(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, lambda_=0.1, max_iter=10, tol=1e-4):
        self.C = C
        self.lambda_ = lambda_
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = None
        self.classes_ = None
        self.le = None
    
    def _loss_function(self, X, y, p):
        entropy = -np.sum(p * np.log(p), axis=1)
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p)) + self.lambda_ * np.mean(entropy)

    def fit(self, X, y):
        # Convert labels to numeric values (0, 1)
        self.le = LabelEncoder()
        y = self.le.fit_transform(y)

        # Initialize the coefficients
        self.coef_ = np.zeros(X.shape[1])
        self.intercept_ = 0.0

        for _ in range(self.max_iter):
            # Compute the logits
            z = self.intercept_ + np.dot(X, self.coef_)
    
            # Compute the probabilities using the sigmoid function
            p = 1 / (1 + np.exp(-z))
    
            # Compute the gradient of the loss function
            gradient = np.dot(X.T, (p - y)) / len(y)
    
            # Compute the Hessian matrix
            hessian = np.dot(X.T * p * (1 - p), X) / len(y)
    
            # Update the coefficients
            self.coef_ -= np.linalg.solve(hessian + self.lambda_ * np.eye(X.shape[1]), gradient)
            self.intercept_ -= np.mean(p - y) / (self.lambda_ + 1e-8)
    
            # Check convergence
            if np.linalg.norm(gradient) < self.tol:
                break

        self.classes_ = self.le.classes_
        return self

    def predict(self, X):
        # Compute the logits
        z = self.intercept_ + np.dot(X, self.coef_)

        # Compute the probabilities using the sigmoid function
        p = 1 / (1 + np.exp(-z))

        # Convert probabilities to class labels (0 or 1)
        y_pred = np.where(p >= 0.5, 1, 0)

        # Convert class labels back to original labels
        y_pred = self.le.inverse_transform(y_pred)

        return y_pred

    def predict_proba(self, X):
        # Compute the logits
        z = self.intercept_ + np.dot(X, self.coef_)

        # Compute the probabilities using the sigmoid function
        p = expit(z)

        # Return the class probabilities
        return np.column_stack((1 - p, p))


In [3]:
df = pd.read_csv("penguins_size.csv")

In [4]:
df = df.dropna()
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE


In [5]:
df = df[df['sex'] != '.']

In [6]:

X = pd.get_dummies(df.drop('species', axis=1), drop_first=True)
y = df['species']


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [8]:
# Feature Scaling
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)


In [9]:
# Create an instance of the custom logistic regression class
log_model = LogisticRegressionWithEntropyRegularization(C=1.0, lambda_=0.1, max_iter=10, tol=1e-4)


In [10]:
# Fit the model to the training data
log_model.fit(scaled_X_train, y_train)

LogisticRegressionWithEntropyRegularization()

In [11]:
# Make predictions on the test data
y_pred = log_model.predict(scaled_X_test)

In [12]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [13]:
# Calculate confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

Confusion Matrix:
[[31  0]
 [ 0 12]]


In [14]:
# Display classification report
classification_report_ = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report_)

Classification Report:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        31
   Chinstrap       1.00      1.00      1.00        12

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



In [15]:
# Make predictions on the training data
y_train_pred = log_model.predict(scaled_X_train)
train_report = classification_report(y_train, y_train_pred)
print("Training Set Performance:")
print(train_report)

Training Set Performance:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00       115
   Chinstrap       1.00      1.00      1.00        56

    accuracy                           1.00       171
   macro avg       1.00      1.00      1.00       171
weighted avg       1.00      1.00      1.00       171



In [16]:
# Make predictions on the test data
y_test_pred = log_model.predict(scaled_X_test)
test_report = classification_report(y_test, y_test_pred)
print("Test Set Performance:")
print(test_report)

Test Set Performance:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        31
   Chinstrap       1.00      1.00      1.00        12

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43

